# Gather data about distances of pubs in Wien for calculating the shortest marathon between pubs in all bezirks

Rules:
* Go over all bezirks in Wien, visit 1 pub/bar in each
* transport  between the pubs: public transport
* Keep order: Bezirk 1 - 23


## info about distance matrix
- https://googlemaps.github.io/google-maps-services-python/docs/index.html#googlemaps.Client.distance_matrix
- https://github.com/googlemaps/google-maps-services-python/blob/master/googlemaps/distance_matrix.py
- https://developers.google.com/maps/documentation/javascript/distancematrix
- https://developers.google.com/maps/documentation/distance-matrix

## How to start with google maps api
- https://developers.google.com/maps/documentation/distance-matrix/cloud-setup


In [1]:
import xml.etree.ElementTree as ET

# get nodes with info from osm file - file generated on https://overpass-turbo.eu/ and exported as "raw OSM data"
def get_nodes(osm_filename):
    tree = ET.parse(osm_filename)
    root = tree.getroot()
    nodes = []
    for i in range(len(root)):
        node = {}
        #print(root[i].tag, root[i].attrib)
        if root[i].tag != "node":
            continue
        node["id"] = root[i].attrib["id"]
        node["lat"] = root[i].attrib["lat"]
        node["lon"] = root[i].attrib["lon"]
        for j in range(len(root[i])):
            attrib = root[i][j].attrib
            node[attrib["k"]] = attrib["v"]
        nodes.append(node)
    return nodes

In [2]:
nodes_bar = get_nodes('wien_bar.osm')
nodes_pub = get_nodes('wien_pub.osm')
all_nodes = nodes_bar + nodes_pub

In [3]:
# see sample
nodes_bar[:3]

[{'id': '84540706',
  'lat': '48.2003245',
  'lon': '16.3929027',
  'addr:city': 'Wien',
  'addr:country': 'AT',
  'addr:housenumber': '82',
  'addr:postcode': '1030',
  'addr:street': 'Landstraßer Hauptstraße',
  'amenity': 'bar',
  'name': 'Ströck Feierabend',
  'opening_hours': 'Mo-Fr 16:00-24:00; Sa 07:00-24:00; Su 07:00-18:00',
  'shop': 'bakery',
  'toilets:wheelchair': 'no',
  'website': 'http://www.stroeck-feierabend.at/',
  'wheelchair': 'yes'},
 {'id': '146328398',
  'lat': '48.2298213',
  'lon': '16.3563982',
  'addr:city': 'Wien',
  'addr:country': 'AT',
  'addr:housenumber': '106',
  'addr:postcode': '1090',
  'addr:street': 'Liechtensteinstraße',
  'amenity': 'bar',
  'description': 'Whiskybar; vormals The Barn',
  'name': 'The Blue Label Bar',
  'opening_hours': '22-',
  'operator': 'Don Enrico'},
 {'id': '309293364',
  'lat': '48.1966120',
  'lon': '16.3651253',
  'addr:city': 'Wien',
  'addr:country': 'AT',
  'addr:housenumber': '24',
  'addr:postcode': '1040',
  'addr

In [4]:
import pandas as pd
df = pd.DataFrame.from_records(all_nodes)
df_postcode = df[df['addr:postcode'].notna()] # leave only those with PLZ so we can identify Bezirk
# shuffle
#df_postcode = df_postcode.sample(frac=1).reset_index(drop=True)
df_postcode.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 415 entries, 0 to 658
Columns: 152 entries, id to darts:type
dtypes: object(152)
memory usage: 496.1+ KB


In [115]:
# Number of bars in each bezirk
possibilities = 1
total_count = 0
previous_count = 1
for i in range(1,24):
    plz="1{:02d}0".format(i)
    count = len(df_postcode[df_postcode['addr:postcode']==plz])
    possibilities *= count
    previous_count = count
    print(f"Bezirk {i} ({plz}): {count}")
    total_count += count
print(f"total count: {total_count}")
print(f"total possibilities {possibilities}")

Bezirk 1 (1010): 80
Bezirk 2 (1020): 22
Bezirk 3 (1030): 22
Bezirk 4 (1040): 16
Bezirk 5 (1050): 13
Bezirk 6 (1060): 29
Bezirk 7 (1070): 45
Bezirk 8 (1080): 28
Bezirk 9 (1090): 32
Bezirk 10 (1100): 10
Bezirk 11 (1110): 4
Bezirk 12 (1120): 3
Bezirk 13 (1130): 6
Bezirk 14 (1140): 4
Bezirk 15 (1150): 19
Bezirk 16 (1160): 23
Bezirk 17 (1170): 10
Bezirk 18 (1180): 10
Bezirk 19 (1190): 12
Bezirk 20 (1200): 6
Bezirk 21 (1210): 3
Bezirk 22 (1220): 9
Bezirk 23 (1230): 4
total count: 410
total possibilities 9216104310659402956800000


In [6]:
# create input format for google maps query
# input - dataframe
# output list of dicts with lat and long
def create_origins(df):
    #df_tmp = df.sample(frac=1).reset_index(drop=True).head(head)
    origins = []
    for row in df.to_dict(orient="records"):
        origin = {"lat" : row["lat"], "lng" : row["lon"]}
        origins.append(origin)
    return origins    

In [7]:
from math import isnan

def get_clean_nodes_dict(df, google_addresses):
    nodes_dict = {}
    for row, address in zip(df.to_dict(orient="records"), google_addresses):
        node = {k: row[k] for k in row if not (isinstance(row[k], float) and isnan(row[k]))}
        node["google_address"] = address
        nodes_dict[row["id"]] = node
    return nodes_dict

In [8]:
def get_from_to_distances(rows, df_origins, df_destinations):
    from_to_distances = []
    origin_ids = [o["id"] for o in df_origins.to_dict(orient="records")]
    destinations_ids = [o["id"] for o in df_destinations.to_dict(orient="records")]
    for origin_id, row in zip(origin_ids, rows):
        # 'rows': [{'elements': [{'distance': {'text': '6.6 km', 'value': 6568}, 'duration': {'text': '32 mins', 'value': 1921},    'status': 'OK'},
        
        for destination_id, dist in zip(destinations_ids, row["elements"]):
            if dist["status"] != "OK":
                print(f"not OK: from {origin_id} to {destination_id}: {dist}")
            from_to_distance = {"from" : origin_id, "to": destination_id, "distance" : dist["distance"]["value"], "duration" : dist["duration"]["value"]}
            from_to_distances.append(from_to_distance)
    return from_to_distances

In [10]:
import googlemaps
# get your own API key!
api = "*****************************"
gmaps = googlemaps.Client(key=api)

import time
from datetime import datetime
from zoneinfo import ZoneInfo

nodes_dict = {}
edges = []


for i in range(1,23):
    plz_origin="1{:02d}0".format(i)
    df_origins = df[df["addr:postcode"]==plz_origin]
    
    plz_destination="1{:02d}0".format(i+1)
    df_destinations = df[df["addr:postcode"]==plz_destination]
    
    print(f"from {plz_origin}({len(df_origins)}) to {plz_destination} ({len(df_destinations)})")
    
    # we need to request with limits
    # limits on requests: https://stackoverflow.com/questions/56405576/why-i-keep-getting-an-error-of-max-elements-exceeded-when-i-dont-overpass-the-d
    requested = 0
    offset_origins = 0
    limit_origins = 10
    limit_destinations = 10
    while True:
        df_origins_tmp = df_origins.iloc[offset_origins: offset_origins+limit_origins]
        if len(df_origins_tmp) == 0:
            break

        offset_origins += limit_origins
        offset_destinations = 0
        while True:
            df_destinations_tmp = df_destinations.iloc[offset_destinations: offset_destinations+limit_destinations]
            if len(df_destinations_tmp) == 0:
                break
            offset_destinations += limit_destinations
            requested += len(df_origins_tmp)*len(df_destinations_tmp)
      
   
            origins = create_origins(df_origins_tmp)
            destinations = create_origins(df_destinations_tmp)
    
            distance_matrix = gmaps.distance_matrix(origins=origins, 
                                                destinations=destinations, 
                                                mode="transit", units="metric", 
                                                transit_routing_preference="fewer_transfers",
                                                departure_time=datetime(2022,10,15,10, tzinfo=ZoneInfo("Europe/Vienna")),
                                                transit_mode=["subway", "train", "tram", "bus"]
            )
            nodes_dict.update(get_clean_nodes_dict(df_origins_tmp, distance_matrix['origin_addresses']))
            nodes_dict.update(get_clean_nodes_dict(df_destinations_tmp, distance_matrix['destination_addresses']))
            
            edges.extend(get_from_to_distances(distance_matrix["rows"], df_origins_tmp, df_destinations_tmp))
            
            print(requested)
            time.sleep(1)


from 1010(80) to 1020 (22)
100
200
220
320
420
440
540
640
660
760
860
880
980
1080
1100
1200
1300
1320
1420
1520
1540
1640
1740
1760
from 1020(22) to 1030 (22)
100
200
220
320
420
440
460
480
484
from 1030(22) to 1040 (16)
100
160
260
320
340
352
from 1040(16) to 1050 (13)
100
130
190
208
from 1050(13) to 1060 (29)
100
200
290
320
350
377
from 1060(29) to 1070 (45)
100
200
300
400
450
550
650
750
850
900
990
1080
1170
1260
1305
from 1070(45) to 1080 (28)
100
200
280
380
480
560
660
760
840
940
1040
1120
1170
1220
1260
from 1080(28) to 1090 (32)
100
200
300
320
420
520
620
640
720
800
880
896
from 1090(32) to 1100 (10)
100
200
300
320
from 1100(10) to 1110 (4)
40
from 1110(4) to 1120 (3)
12
from 1120(3) to 1130 (6)
18
from 1130(6) to 1140 (4)
24
from 1140(4) to 1150 (19)
40
76
from 1150(19) to 1160 (23)
100
200
230
320
410
437
from 1160(23) to 1170 (10)
100
200
230
from 1170(10) to 1180 (10)
100
from 1180(10) to 1190 (12)
100
120
from 1190(12) to 1200 (6)
60
72
from 1200(6) to 1210 (3)

In [30]:
len(edges)

8172

In [12]:
nodes = [n for n in nodes_dict.values()]

final_dict = {
    "nodes" : nodes,
    "edges" : edges
}

import json

# save the json file
with open("wien_bar_distances-public_transport.json", "wt") as f:
    json.dump(final_dict, f, indent=4, ensure_ascii=False)

In [19]:
# add edges to graph

import networkx as nx
import matplotlib.pyplot as plt

g = nx.Graph()

for edge in edges:
    g.add_edge(edge["from"],edge["to"],weight=edge["duration"])
    

pos = nx.spring_layout(g, k=10)

In [23]:
def get_duration(f, t):
    for edge in edges:
        if edge["from"] == f and edge["to"] == t:
            return edge["duration"]      

In [107]:
# find the shortest path

plz_origin="1{:02d}0".format(1)
df_origins = df[df["addr:postcode"]==plz_origin]
    
plz_destination="1{:02d}0".format(23)
df_destinations = df[df["addr:postcode"]==plz_destination]

origin_ids = [o["id"] for o in df_origins.to_dict(orient="records")]
destinations_ids = [o["id"] for o in df_destinations.to_dict(orient="records")]

minimum = 999999
min_path = []
for o in origin_ids:
    for d in destinations_ids:
        path = nx.shortest_path(g, source=f"{o}",target=f"{d}", weight='weight')
        duration_total = 0
        previous = path[0]
        for i in range(1,len(path)):
            node_previous = path[i-1]
            node_current = path[i]
            duration = get_duration(node_previous, node_current)
            #print(f"from {nodes_dict[node_previous]['name']} {nodes_dict[node_previous]['addr:postcode']} to {nodes_dict[node_current]['name']} {nodes_dict[node_current]['addr:postcode']} takes {duration} min")
            duration_total += duration
        #print(duration_total/60)
        if duration_total < minimum:
            #print(f"from {nodes_dict[path[0]]['name']} {nodes_dict[path[0]]['addr:postcode']} to {nodes_dict[path[-1]]['name']} {nodes_dict[path[-1]]['addr:postcode']} duration total min: {duration_total/60}")
            minimum = duration_total
            min_path = path

duration_total = 0
previous = path[0]
for i in range(1,len(min_path)):
    node_previous = min_path[i-1]
    node_current = min_path[i]
    duration = get_duration(node_previous, node_current)/60
    print(f"from {nodes_dict[node_previous]['name']} {nodes_dict[node_previous]['addr:postcode']} to {nodes_dict[node_current]['name']} {nodes_dict[node_current]['addr:postcode']} takes {duration} min")
    duration_total += duration
print(f"duration total min: {duration_total}")            

from Fledermaus 1010 to Kerstin 1020 takes 7.116666666666666 min
from Kerstin 1020 to 's Stammbeisl 1030 takes 7.983333333333333 min
from 's Stammbeisl 1030 to Cafe Nest 1040 takes 13.816666666666666 min
from Cafe Nest 1040 to Pub Klemo 1050 takes 3.3833333333333333 min
from Pub Klemo 1050 to Miranda 1060 takes 8.25 min
from Miranda 1060 to DAS Biero 1070 takes 8.316666666666666 min
from DAS Biero 1070 to Brot und Spiele 1080 takes 8.0 min
from Brot und Spiele 1080 to Wachauer Weinstube 1090 takes 7.766666666666667 min
from Wachauer Weinstube 1090 to Aurora Rooftop Bar 1100 takes 24.616666666666667 min
from Aurora Rooftop Bar 1100 to Café Frog 1110 takes 13.3 min
from Café Frog 1110 to Golden Harp Kaffeehaus 1120 takes 22.25 min
from Golden Harp Kaffeehaus 1120 to What's up 1130 takes 23.05 min
from What's up 1130 to Schani's Beisl 1140 takes 10.283333333333333 min
from Schani's Beisl 1140 to Travelshack Vienna 1150 takes 18.5 min
from Travelshack Vienna 1150 to Velvet Palace 1160 take

In [108]:
# display markers
import folium

node0 = nodes_dict[min_path[0]]
m = folium.Map(location=[float(node0['lat']),float(node0['lon'])])

#draw nodes
for i in range(len(min_path)):
    node = nodes_dict[min_path[i]]
    folium.Marker(
    [float(node['lat']),float(node['lon'])], tooltip=f"<i>{i+1}: {node['name']}</i>").add_to(m)

# draw edges
for i in range(1,len(min_path)):
    node_previous = min_path[i-1]
    node_current = min_path[i]
    loc=[(float(nodes_dict[node_previous]['lat']), float(nodes_dict[node_previous]['lon'])),(float(nodes_dict[node_current]['lat']), float(nodes_dict[node_current]['lon'])) ]
    duration = get_duration(node_previous, node_current)/60
    folium.PolyLine(loc, color='red', weight=5,opacity=0.8, tooltip=duration).add_to(m)    
    
m    

In [110]:
m.save('map.html')